# 행렬 요인화 (Matrix Factorization )

### Overview

- 추천을 위한 다양한 알고리즘을 분류를 해보면 크게 메모리 기반(memory-based)과 모델 기반(model-based)으로 나눌 수 있다. 
- 메모리 기반 알고리즘은 추천을 위한 데이터를 모두 메모리에 가지고 있으면서 추천이 필요할 때마다 이 데이터를 사용해서 계산을 해서 추천하는 방식을 말한다. CF가 대표적인 메모리 기반 알고리즘이라고 할 수 있다. 
- 모델 기반 추천은 데이터로부터 추천을 위한 모델을 구성한 후에 이 모델만 저장하고, 실제 추천을 할 때에는 이 모델을 사용해서 추천하는 방식을 말한다. 행렬 요인화 (Matrix factorization) 방식이 대표적인 모델 기반 추천 알고리즘이다. 또한 딥러닝 방식의 추천도 데이터는 신경망 학습에 사용ㅇ되고 에측은 학습된 신경망을 가지고 한다는 점에서 모델 기반 추천 알고리즘이라고 할 수 있다.

- 메모리 기반 추천은 모든 데이터를 메모리에 저장하고 있기 때문에 원래 데이터를 충실하게 사용하는 장점이 있지만 대양의 데이터를 다뤄야 하는 사용 사이트에서는 계산시간이 너무 오래 걸린다는 단점이 있다. 
- 이에 비해 모델 기반 추언 방식은 원래 데이터는 모형을 만드는 데만 사용하고 일단 모델이 만들어지면 원래 데이터는 사용하지 않기 대문에 대규모 사용 사이트에서는 필요한 빠른 반응이 가능하지만 모델을 만드는 과정에서 많은 계산이 필요하다는 단점이 있다 

- 일반적으로 메모리 기반 추천은 개별 사용자의 데이터에 집중하는데 비해, 모델 기반 추천은 전체 사용자의 평가 패턴으로부터 모델을 구성하기 때문에 데이터가 가지고 있는 약한 신호(weak signal)도 더 잘 잡아내는 장점이 있다. 여기서 말하는 약한 신호라는 것은 개별 사용자의 행동분석에서는 잘 드러나지 않는 패턴을 말한다. 예를 들어 소수의 사용자가 소수의 영화에 대해서만 특정한 평가 패턴이 있는 경우, 개별 사용자나 개별 아이템에 집중하는 메모리 기반 알고리즘은 이것을 잡아내기 쉽지 않지만 전체 데이터를 대상으로 모델을 구성하는 모델 기반 추천은 이것을 더 잘 잡아 낼 수 있다는 것이다. 

### SGD(Stochastic Gradient Decent)를 사용한 MF 알고리즘

- 어떤 도메인에 대해서 사용자와 아이템의 특성을 잘 설명할 수 있는 K개의 요인이 존재하고, 각 사용자와 아이템의 K개 요인에 대한 측정값을 알아 낼 수 있다면 모든 사용자의 모든 아이템에 대한 예측 평점을 계산할 수 있다. 만일 사용자가 어떤 영화에 대해서 실제 평점을 부여하였다면, 그 실제 평점과 예상 평점의 차이가 정확도 이다 

In [1]:
import numpy as np
import pandas as pd

from sklearn.utils import shuffle

In [2]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('data/u.data', names=r_cols,  sep='\t',encoding='latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)            # timestamp 제거
ratings

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [3]:
# train test 분리

TRAIN_SIZE = 0.75 # train 75% / test 25%
ratings = shuffle(ratings, random_state=1) # ratings를 random sampling / ratings는 사용자 -영화- 평점이 하나의 세트로 되어 있음
cutoff = int(TRAIN_SIZE * len(ratings)) # Train_size의 비율에 해당하는 데이터가 몇 개인지 계산 
ratings_train = ratings.iloc[:cutoff] # train_size에 해당하는 데이터를 슬라이싱으로 앞에 데이터로 저장
ratings_test = ratings.iloc[cutoff:] #  슬라이싱으로 뒤에 데이터로 저장
# ratings = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0) # full matrix로 반환

In [4]:
# MF class
class MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R = np.array(ratings) # dataframe 형식으로 전달된 평점을 array로 변경
        # user_id, item_id를 R의 index와 매핑하기 위한 dictionary 생성
        item_id_index = [] # 변수 최기화 
        index_item_id = [] # 변수 초기화 
        for i, one_id in enumerate(ratings): # 넘겨 받은 데이터 ratings의 각 아이템에 대해서 아래 작업 수행
            item_id_index.append([one_id, i]) # 아이디를 인덱스로 매핑해 주는 item_id_index에 현재 아이템의 아이디와 인덱스를 저장 
            index_item_id.append([i, one_id]) # 반대로 인덱스를 아이디로 매핑해주는 맵 index_item_id에 현재 아이템의 인덱스와 아이들 저장 
        self.item_id_index = dict(item_id_index) # dict 형태로 변환
        self.index_item_id = dict(index_item_id)        
        # 위의 작업을 사용자 아이디에 대해서 실행 
        user_id_index = [] 
        index_user_id = []
        for i, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)
        self.index_user_id = dict(index_user_id)
        
        self.num_users, self.num_items = np.shape(self.R) # 사용자 수와(num_users)와 아이템 수(num_items)를 받아온다 
        self.K = K # 잠재요인(latent factor)의 수 
        self.alpha = alpha # learning rate
        self.beta = beta # 정규화 계수
        self.iterations = iterations # 반복횟수
        self.verbose = verbose # 중간 학습 과정을 출력
    
    # Root Mean Squared Error (RMSE) 계산
    def rmse(self):
        xs, ys = self.R.nonzero() # R에서 평점이 있는 (0이 아닌) 요소의 인덱스를 가져온다 
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y) # 사용자 x, 아이템 y에 대해서 평점 예측치 
            self.predictions.append(prediction) #
            self.errors.append(self.R[x, y] - prediction) # 실제값(R)과 예측값의 차이(errors) 계산해서 오차값 리스트에 추가 
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))
       
    # Rating prediction for user i and item j
    # 평점 에측값을 구하는 함수 
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j) # 사용자 i, 아이템 j에 대한 평점 예측치를 구한다 
            e = (r - prediction) # 실제 평점과 비교해서 오차를 구함

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])
            
    # Test set을 선정
    def set_test(self, ratings_test):
        test_set = []
        for i in range(len(ratings_test)):      # test 데이터에서 각 (사용자-아이템-평점)에 대해서 작업 반복
            x = self.user_id_index[ratings_test.iloc[i, 0]] # 현재 사용자의 인덱스를 user_id_index에서 받아온다 
            y = self.item_id_index[ratings_test.iloc[i, 1]] # 현재 아이템의 인덱스를 item_id_index에서 받아온다
            z = ratings_test.iloc[i, 2] # 사용자 -아이템의 평점을 받아 온다 
            test_set.append([x, y, z])
            self.R[x, y] = 0                    # Setting test set ratings to 0
            # 0으로 지우는 이유는 R을 ㅏㅅ용해서 MF 모델을 학습을 하기 때문에 test_set은 R에서 제거해야 하기 때문이다
        self.test_set = test_set
        return test_set                         # Return test set

    # Test set의 RMSE 계산
    def test_rmse(self):
        error = 0
        for one_set in self.test_set: # test set에 대해서 각 (사용자-아이템-평점)에 대해서 작업 반복
            predicted = self.get_prediction(one_set[0], one_set[1]) # 예측 평점
            error += pow(one_set[2] - predicted, 2) 
            # 현재 (사용자-아이템-평점)에서의 평점 실제값이고 predicted가 예측치이므로 error에는 오차를 제곱한 것을 누적한 값이 저장 
        return np.sqrt(error/len(self.test_set))

    # Training 하면서 test set의 정확도를 계산
    def test(self):
        # Initializing user-feature and item-feature matrix
        # P , Q행렬을 임의 값으로 채움, 평균 0, 표준편차 1/k 인 정규 분포의 난수 발생
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K)) 
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K)) 


        # Initializing the bias terms
        # bias 
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()]) # 전체 평균 b

        # List of training samples
        rows, columns = self.R.nonzero() # 평점 행렬 R 중에서 평점이 있는 (0이 아닌) 요소의 인덱스를 가져온다 
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]
        # SGD를 적용할 대상, 즉 평점이 있는 요소의 인덱스와 평점을 리스트로 만들어 sample에 저장 
        
        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples) # sample를 임의로 섞는다.
            '''
            임의로 섞는 이유는 다른 기계학습 알고리즘과 마찬가지로 SGD를 어디서 시작하느냐에 따라 수렴의 속도가 달라질 수 있기 때문에 
            매 반복마다 다양한 시작점에서 출발하기 위함이다 
            '''
            self.sgd()
            rmse1 = self.rmse()
            rmse2 = self.test_rmse()
            training_process.append((i+1, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d : Train RMSE = %.4f , Test RMSE = %.4f" % (i+1, rmse1, rmse2)) # change
        return training_process

    # Ratings for given user_id and item_id
    # 주어진 user_id와 item_id에 대한 예측지를 계산해서 돌려준다
    def get_one_prediction(self, user_id, item_id):
        return self.get_prediction(self.user_id_index[user_id], self.item_id_index[item_id])

    # Full user-movie rating matrix
    # 모든 사용자의 모든 아이템에 대한 예측치(full matrix)를 계산해서 돌려준다 
    def full_prediction(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_d[np.newaxis,:] + self.P.dot(self.Q.T)

In [5]:
# 전체 데이터 사용 MF
# dataframe 형식으로 되어 있는 rating 데이터를 full matrix로 변환한다.
R_temp = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
mf = MF(R_temp, K=30, alpha=0.001, beta=0.02, iterations=100, verbose=True)
test_set = mf.set_test(ratings_test)
result = mf.test()

Iteration: 10 : Train RMSE = 0.9659 , Test RMSE = 0.9834
Iteration: 20 : Train RMSE = 0.9410 , Test RMSE = 0.9645
Iteration: 30 : Train RMSE = 0.9298 , Test RMSE = 0.9567
Iteration: 40 : Train RMSE = 0.9231 , Test RMSE = 0.9524
Iteration: 50 : Train RMSE = 0.9184 , Test RMSE = 0.9498
Iteration: 60 : Train RMSE = 0.9145 , Test RMSE = 0.9480
Iteration: 70 : Train RMSE = 0.9109 , Test RMSE = 0.9467
Iteration: 80 : Train RMSE = 0.9071 , Test RMSE = 0.9455
Iteration: 90 : Train RMSE = 0.9024 , Test RMSE = 0.9444
Iteration: 100 : Train RMSE = 0.8965 , Test RMSE = 0.9430


In [6]:
# Printing predictions
print(mf.full_prediction())
print(mf.get_one_prediction(1, 2))

[[3.86421452 3.42751269 3.08344786 ... 3.32550996 3.50005603 3.42613165]
 [3.95328502 3.49186394 3.16331664 ... 3.42482036 3.55061759 3.5396195 ]
 [3.34888768 2.87242725 2.55290044 ... 2.8314933  2.92935429 2.94663308]
 ...
 [4.19563198 3.76188387 3.41916886 ... 3.70778215 3.82511852 3.82764029]
 [4.36617162 3.89123327 3.55698476 ... 3.82846504 3.94844264 3.94726942]
 [3.76587754 3.3990825  3.03310164 ... 3.29354246 3.40595532 3.41548068]]
3.4275126932466846


- 모든 예측 평점에 대한 full matrix가 출력됨을 볼 수 있다. 마지막은 약 3.42이라는 숫자는 user_id가 1인 사용자의 item_id 2번 아이템에 대한 예측치 이다